In [0]:
!git clone https://github.com/KodeWil/final_project_AI_csv_Data.git

In [0]:
#Dependencies
# %reset
import keras
from keras.models import Sequential
from keras.layers import Dense# Neural network
import pandas as pd 
import glob 
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
import copy
from google.colab import files as fl

In [0]:
def addData(files: str) -> list:
    allCsv = []
    for file in files:
      temp = open(file)
      tempDf = pd.read_csv(file)
      allCsv.append(tempDf)
    return allCsv



In [0]:
def printAll(df):
  pd.set_option('display.max_rows', None)
  print(df)

In [24]:
frames = []

#Create a list ('frames') with four nested lists, one per sensor. 
#Each nested list have all the samples (DF) for that sensor.
%cd /content/final_project_AI_csv_Data/phone/accel
files = glob.glob('*.csv')
files.sort()
frames.append(addData(files))
%cd /content/final_project_AI_csv_Data/phone/gyro
files = glob.glob('*.csv')
files.sort()
frames.append(addData(files))
%cd /content/final_project_AI_csv_Data/watch/accel
files = glob.glob('*.csv')
files.sort()
frames.append(addData(files))
%cd /content/final_project_AI_csv_Data/watch/gyro
files = glob.glob('*.csv')
files.sort()
frames.append(addData(files))
print(len(frames))
# 0 in frames = /phone/accel
# 1 in frames = /phone/gyro
# 2 in frames = /watch/accel
# 3 in frames = /watch/gyro

/content/final_project_AI_csv_Data/phone/accel
/content/final_project_AI_csv_Data/phone/gyro
/content/final_project_AI_csv_Data/watch/accel
/content/final_project_AI_csv_Data/watch/gyro
4


In [0]:
def cutSamples(frames1, frames2):
  for i in range (0,50):
    samples1 = frames1[i].shape[0]
    samples2 = frames2[i].shape[0]
    if(samples1 > samples2):
      toDrop = list(range(samples2, samples1))
      frames1[i].drop(frames1[i].index[[toDrop]], inplace = True)
    elif(samples2 > samples1):
      toDrop = list(range(samples1, samples2))
      frames2[i].drop(frames2[i].index[[toDrop]], inplace = True) 


In [0]:
def kerasSequiential(X, y):
  InpDim = X.shape[1]
  model = Sequential()
  model.add(Dense(int(InpDim*2), input_dim=InpDim, activation="relu"))
  model.add(Dense(64, activation="relu"))
  model.add(Dense(18, activation="softmax"))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  history = model.fit(X, y, epochs=30, batch_size=20)
  print("")
  print("")
  print("")
  return model

In [0]:
#Split into train and test data, encode "ACTIVITY" column into a numerical column
def dataTransformation(frame, columns: bool):
  ohe = OneHotEncoder()
  sc = StandardScaler()
  if(columns):
    y = frame["ACTIVITY"]
    X = frame.drop(columns = ["ACTIVITY", "class"])
    X = X.values
    X = sc.fit_transform(X)
    y = y.values
    y = y.reshape(-1,1)
    y = ohe.fit_transform(y).toarray()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    return (kerasSequiential(X_train, y_train), X_test, y_test)
  else:
    y = frame[0]
    y = y.values
    y = y.reshape(-1,1)
    y = ohe.fit_transform(y).toarray()
    X = frame.drop(columns = [0,91,93,182])
    X = X.values
    X = sc.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    return (kerasSequiential(X_train, y_train), X_test, y_test)






In [27]:
# Concatenate all phone Dataframes in a single One
sensors = []
phoneAccel = pd.concat(frames[0])
sensors.append(phoneAccel)
phoneGyro = pd.concat(frames[1])
sensors.append(phoneGyro)
watchAccel = pd.concat(frames[2])
sensors.append(watchAccel)
watchGyro = pd.concat(frames[3])
sensors.append(watchGyro)


#Cut samples to match with the number of samples in the other sensor (accel and gyro)
cutSamples(frames[0], frames[1]) #Phone sensors
cutSamples(frames[2], frames[3]) #Watch sensors

#Concat all individual samples for a single sensor in one data frame. 
phoneAccel = pd.concat(frames[0], ignore_index=True)
phoneGyro = pd.concat(frames[1], ignore_index=True)
watchAccel = pd.concat(frames[2], ignore_index=True)
watchGyro = pd.concat(frames[3], ignore_index=True)

#Create a new data frame with both sensors for both devices.
phone = pd.concat([phoneAccel, phoneGyro], ignore_index=True, axis = 1)
phoneIndex = phone[phone[0] != phone[93]].index
phone.drop(phoneIndex, inplace=True)
watch = pd.concat([watchAccel, watchGyro], ignore_index=True, axis = 1)
watchIndex = watch[watch[0] != watch[93]].index
watch.drop(watchIndex, inplace=True)

models = []
for sensor in sensors:
  models.append(dataTransformation(sensor, True))

models.append(dataTransformation(phone, False))
models.append(dataTransformation(watch, False))

names = ["Phone Accel", "Phone Gyro" , "Watch Accel", "Watch Gyro", "Phone", "Watch" ]
print("")
print("")
print("Models' scores")
print("")
print("")

for i, model in  enumerate(models):
  score, acc = model[0].evaluate(model[1], model[2],
                            batch_size=20)
  print("")
  print("The accuracy for " + names[i]+ " is: " + str(acc))
  print("")


Epoch 1/30
12096/12096 [==============================] - 1s 70us/step - loss: 1.7053 - accuracy: 0.4334
Epoch 2/30
12096/12096 [==============================] - 1s 64us/step - loss: 1.2320 - accuracy: 0.5899
Epoch 3/30
12096/12096 [==============================] - 1s 73us/step - loss: 1.0370 - accuracy: 0.6577
Epoch 4/30
12096/12096 [==============================] - 1s 66us/step - loss: 0.9023 - accuracy: 0.7044
Epoch 5/30
12096/12096 [==============================] - 1s 63us/step - loss: 0.8129 - accuracy: 0.7309
Epoch 6/30
12096/12096 [==============================] - 1s 63us/step - loss: 0.7464 - accuracy: 0.7581
Epoch 7/30
12096/12096 [==============================] - 1s 63us/step - loss: 0.6945 - accuracy: 0.7707
Epoch 8/30
12096/12096 [==============================] - 1s 64us/step - loss: 0.6515 - accuracy: 0.7865
Epoch 9/30
12096/12096 [==============================] - 1s 62us/step - loss: 0.6007 - accuracy: 0.7995
Epoch 10/30
12096/12096 [==============================